<a href="https://colab.research.google.com/github/bahkali/SkinCancerClassifier/blob/main/SkinCancerClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classifier Skin Cancer 
 using dataset from ISIC 2019 to determine if image of skin is either bengin (Unharmful) or if it is maligmant ( actually cancer). 
## 1. Purpose
The goal is to implement the multi-classifier model to Flutter app for : Android or Ios  to identifie 9 skin cancer .

## 2. Data
Dataset:
https://www.kaggle.com/nodoubttome/skin-cancer9-classesisic


9 floder:
  - actinic keratosis
  - basal cell carcinoma
  - dermatofibroma
  - melanoma
  - nevus
  - pigmented benign keratosis seborrheic  keratosis
  - squamous cell carcinoma
  - vascular lesion





In [1]:
# import Libraries
import tensorflow as tf
import tensorflow_datasets as tfds
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import  Input, Model, regularizers
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Dropout, Dense, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# Import Resnet50 
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input


## Upload Dataset and Preprocessing Images 


1. Take an images filepath
2. create a image generator
3. write a function to prepare the datases which: shuffle, batched and prefetch for faster

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# get file path
train_path = "/content/drive/MyDrive/Colab-Notebooks/end-to-end-skin-cancer/datasets/skin-cancer/training"
test_path = "/content/drive/MyDrive/Colab-Notebooks/end-to-end-skin-cancer/datasets/skin-cancer/test"

In [5]:
# Hyperparameters
img_height = 224
img_width = 224
img_size = [224, 224]
batch_size = 64
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [6]:
# Load images from drive and split it to training and validation

ds_train = tf.keras.preprocessing.image_dataset_from_directory(
    train_path,
    labels='inferred',
    label_mode ='categorical',
    color_mode='rgb',
    image_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle=True,
    seed=42,
    validation_split=0.2,
    subset='training'
)
ds_validation = tf.keras.preprocessing.image_dataset_from_directory(
    train_path,
    labels='inferred',
    label_mode ='categorical',
    color_mode='rgb',
    image_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle=True,
    seed=42,
    validation_split=0.2,
    subset='validation'
)
ds_test = tf.keras.preprocessing.image_dataset_from_directory(
    test_path,
    labels = 'inferred',
    label_mode='categorical',
    image_size=(img_height, img_width),
    batch_size=batch_size,
    color_mode='rgb',
    shuffle=True,
    seed=42,
)
# train_gen = ImageDataGenerator(rescale=1./255,)
# ds_train = train_gen.flow_from_directory(train_path,
#                                          batch_size=batch_size,
#                                          target_size=(img_height, img_height),
#                                          class_mode='categorical',
#                                          color_mode='rgb',
#                                          shuffle=True,
#                                          subset='training',)


Found 2239 files belonging to 9 classes.
Using 1792 files for training.
Found 2239 files belonging to 9 classes.
Using 447 files for validation.
Found 118 files belonging to 9 classes.


In [8]:
# Image Augmentation function for better result
@tf.function
def preprocess_img(image, label):
  image = tf.image.resize(image, [224, 224])
  image = tf.cast(image, tf.float32) / 255.0 # Normalize
  image = tf.image.random_brightness( image , max_delta=0.05)
  # image = tf.image.random_flip_left_right(image)
  image = preprocess_input(image)
  return image, label

def preprocess_img_val(image, label):
  image = tf.image.resize(image, [224, 224])
  image = preprocess_input(image)
  return image, label

In [9]:
ds_train = ds_train.map(preprocess_img, num_parallel_calls=AUTOTUNE).cache()
ds_train = ds_train.shuffle(1000).batch(batch_size)
ds_train = ds_train.prefetch(AUTOTUNE)

ds_validation = ds_validation.map(preprocess_img_val, num_parallel_calls=AUTOTUNE).cache()
ds_validation = ds_validation.batch(batch_size)
ds_validation = ds_validation.prefetch(AUTOTUNE)

ds_test = ds_test.map(preprocess_img_val, num_parallel_calls=AUTOTUNE)
ds_test = ds_test.batch(64).prefetch(AUTOTUNE)


In [ ]:
plt.figure(figsize=(15,15))
for i in range(9):
  ax = plt.subplot(3,3,i+1)
  for datapoint in tfds.as_numpy(ds_train.take(1)):
    plt.imshow(datapoint[0][0].astype('uint8'))
    plt.axis('off')
plt.show()

## Building a Model

We build a model using Functional API and use from TensorFlow keras application Resnet50 https://www.tensorflow.org/api_docs/python/tf/keras/applications/ResNet50



In [11]:
resnet50 = ResNet50(input_shape=img_size+[3], weights='imagenet', include_top=False)
for layer in resnet50.layers:
  layer.trainable = False
resnet50.summary()

94773248/94765736 [==============================] - 0s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
__________________________

In [12]:
input = Flatten()(resnet50.output)
hidden = Dense(64, activation='relu')(input)
output = Dense(9, activation='softmax')(hidden)
model = Model(inputs=resnet50.input, outputs = output)

model.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [13]:
# Metrics = [
#          tf.keras.metrics.CategoricalCrossentropy(),
#          tf.keras.metrics.Precision(n)  
# ]
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss= tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [14]:
history = model.fit(ds_train,
                    epochs=5,
                    validation_data=ds_validation,
                    steps_per_epoch= 800,
                    validation_steps= 800,
                    verbose=5
                    )

Epoch 1/5


ValueError: ignored